# Quantum Simulation of a free Schrödinger Wave
This implementation is based upon the paper **"Simulating Quantum Mechanics on a Quantum Computer"** attached in /research

In [31]:
from qiskit import QuantumCircuit as QC
from qiskit import transpile
from qiskit.circuit.library import iSwapGate
import numpy as np
from qiskit_aer import QasmSimulator

## 1. State Preparation
Using a recursive controlled rotation algorithm, we match the amplitudes of each qubit state with the given discretized values of the wave equation

###  1.1 Mock 3 values Wave amplitudes

In [32]:
psi = np.array([0.5, 0.7, 0.5], dtype=complex) # complex wavefunction
psi /= np.linalg.norm(psi)  # Normalize the wavefunction
print(psi)

[1.11111110e-04+0.j 9.99999988e-01+0.j 1.11111110e-04+0.j]


### 1.2 Prepare Quantum Circuit with 3 qubits and 3 classical bits as registers
As discussed in the paper, each qubit represents a discrete position.

So n positions require n qubits, not the most optimal solution.

In [33]:
qc = QC(3, 3)

### 1.3 Rotating into correct state vector

#### 1.3.1 - Probability Tree

```plaintext
Amplitude Tree for |ψ> = c1|100> + c2|010> + c3|001>

Start: |000>  (amplitude = 1)
│
├─ R_y(θ1) on q0
│   ├─ |100> :  sin(θ1/2) = c1
│   └─ |000> :  cos(θ1/2) = √(1 - c1²)
│
├─ Controlled R_y(θ2) on q1  (control q0 = 0)
│   ├─ |010> :  √(1 - c1²) * sin(θ2/2) = c2
│   └─ |000> :  √(1 - c1²) * cos(θ2/2) = √(1 - c1² - c2²)
│
└─ Controlled R_y(θ3) on q2  (controls q0 = 0, q1 = 0)
    ├─ |001> :  √(1 - c1² - c2²) * sin(θ3/2) = c3
    └─ |000> :  √(1 - c1² - c2²) * cos(θ3/2) = 0
```

Start: |000>

1. Rotate qubit 0: 
$|\psi\rangle' = cos(\theta_1/2)*|000\rangle+sin(\theta_1/2)*|100\rangle$

```plaintext
    ry(theta, qubit, label=None)
```

In [34]:
theta1 = 2*np.arcsin(psi[0]).real
qc.ry(theta1, 0)

2. Controlled rotation of qubit 1, ctrl being qubit 0:

```plaintext
    cry(theta, control_qubit, target_qubit, label=None, ctrl_state=None)
```

In [35]:
theta2 = 2*np.arcsin(psi[1]/np.sqrt(1-psi[0]**2)).real
qc.cry(theta2, 0, 1, ctrl_state=0)

3. Multi controlled rotation of qubit 2, ctrl being qubits 0 and 1

Visualization:
```plaintext
| Original (q0, q1) | After first Xs | Controlled gate acts? | After last Xs (restored) |
|--------------------|----------------|------------------------|---------------------------|
| 00                 | 11             | ✅ Yes                 | 00                        |
| 01                 | 10             | ❌ No                  | 01                        |
| 10                 | 01             | ❌ No                  | 10                        |
| 11                 | 00             | ❌ No                  | 11                        |

```

In [36]:
theta3 = 2*np.arcsin(psi[2]/np.sqrt(1-psi[0]**2-psi[1]**2)).real
qc.x(0)
qc.x(1)
qc.mcry(theta3, [0, 1], 2)
qc.x(0)
qc.x(1)

## 2. Unitary transformation in free space

$ U(t) = exp(-i*H*\delta t) $, with H being the Hamiltonian and $ \hbar \ = 1 $

In [37]:
b = 0.8660254037844386
a = 0.49999999999999994j
U = np.matrix([[1, 0, 0, 0],
               [0, b, a, 0],
               [0, a, b, 1],
               [0, 0, 0, 1]])

We define:

$$
b = \cos(\theta), \quad a = i \sin(\theta)
$$

The two-qubit state is written as:

$$
|\psi\rangle = c_{00}|00\rangle + c_{01}|01\rangle + c_{10}|10\rangle + c_{11}|11\rangle
$$

After applying \(U\), the coefficients transform as:

$$
\begin{aligned}
c'_{00} &= c_{00},\\[4pt]
\begin{pmatrix}
c'_{01}\\
c'_{10}
\end{pmatrix}
&=
\begin{pmatrix}
\cos\theta & i\sin\theta\\
i\sin\theta & \cos\theta
\end{pmatrix}
\begin{pmatrix}
c_{01}\\
c_{10}
\end{pmatrix},\\[4pt]
c'_{11} &= c_{11}.
\end{aligned}
$$

The corresponding matrix representation (in the basis $|00\rangle, |01\rangle, |10\rangle, |11\rangle$) is:

$$
U =
\left(
\begin{array}{c|cccc}
 & |00\rangle & |01\rangle & |10\rangle & |11\rangle \\ \hline
|00\rangle & 1 & 0 & 0 & 0 \\
|01\rangle & 0 & \cos\theta & i\sin\theta & 0 \\
|10\rangle & 0 & i\sin\theta & \cos\theta & 0 \\
|11\rangle & 0 & 0 & 0 & 1
\end{array}
\right)
$$


In [38]:
def time_transform(circuit: QC, U: np.matrix):
    # theta = np.arccos(U[1,1].real)
    theta = np.pi/2  # Example fixed angle for demonstration
    n = range(circuit.num_qubits - 1)
    for i in n:
        circuit.rxx(-theta, circuit.qubits[i], circuit.qubits[i+1])
        circuit.ryy(-theta, circuit.qubits[i], circuit.qubits[i+1])
    return circuit

## 3. Time Evolution

In [39]:
for i in range(np.pow(10, 5)):
    qc = time_transform(qc, U)

## 4. Quantum Simulator - Measurement

In [40]:
qc.measure([0, 1, 2], [0, 1, 2])

In [41]:
backend = QasmSimulator(method='statevector')
job = backend.run(qc, shots=np.pow(2,20))
result = job.result()

counts = result.get_counts()
print(counts)

{'001': 1048576}


## 5. Map into np array

In [42]:
# --- total number of shots
shots = sum(counts.values())

# --- all 3-qubit basis states (2^3 = 8)
states = ['000','001','010','011','100','101','110','111']

# --- convert counts → probabilities
pvec = np.array([counts.get(s, 0) / shots for s in states])

# --- make a matching NumPy array of state labels
states_arr = np.array(states)

# --- eliminate zero components
mask = pvec > 0
pvec_nonzero = pvec[mask]
states_nonzero = states_arr[mask]

# --- (optional) compute amplitudes from probabilities
amps_nonzero = np.sqrt(pvec_nonzero)

# --- pretty print results
print("Non-zero states:", states_nonzero)
print("Probabilities:", pvec_nonzero)
print("Amplitudes:", amps_nonzero)

# --- optional: combine into a dictionary
filtered = dict(zip(states_nonzero, pvec_nonzero))
filtered_amps = dict(zip(states_nonzero, amps_nonzero))
print("Dictionary (probabilities):", filtered)
print("Dictionary (amplitudes):", filtered_amps)

Non-zero states: ['001']
Probabilities: [1.]
Amplitudes: [1.]
Dictionary (probabilities): {np.str_('001'): np.float64(1.0)}
Dictionary (amplitudes): {np.str_('001'): np.float64(1.0)}


## 6. Test with iSwap given theta = pi/6

1. Reinitialize qubit states

In [43]:
qc_test = QC(3, 3)

theta1 = 2*np.arcsin(psi[0]).real
qc_test.ry(theta1, 0)

theta2 = 2*np.arcsin(psi[1]/np.sqrt(1-psi[0]**2)).real
qc_test.cry(theta2, 0, 1, ctrl_state=0)

theta3 = 2*np.arcsin(psi[2]/np.sqrt(1-psi[0]**2-psi[1]**2)).real
qc_test.x(0)
qc_test.x(1)
qc_test.mcry(theta3, [0, 1], 2)
qc_test.x(0)
qc_test.x(1)

2. Transform using iSwap gates

    IBM iSwap Docs: https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.circuit.library.iSwapGate

In [44]:
n = range(qc_test.num_qubits - 1)
for i in range(2):
    for k in n:
        qc_test.iswap(k, k+1)

3. Measure

In [45]:
qc_test.measure([0, 1, 2], [0, 1, 2])

backend_test = QasmSimulator(method='statevector')
qc_test = transpile(qc_test, backend=backend_test, basis_gates=['u3', 'cx'])
job_test = backend.run(qc_test, shots=np.pow(2,20))
result_test = job_test.result()

counts_test = result_test.get_counts()
print(counts)

/opt/anaconda3/envs/guppylang/lib/python3.13/site-packages/qiskit/compiler/transpiler.py:269: UserWarning: Providing `coupling_map` and/or `basis_gates` along with `backend` is not recommended, as this will invalidate the backend's gate durations and error rates.
  pm = generate_preset_pass_manager(


{'001': 1048576}
